<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

Original GitHub link https://github.com/microsoft/recommenders/blob/master/examples/02_model_content_based_filtering/mmlspark_lightgbm_criteo.ipynb

# Content-Based Personalization with LightGBM on Spark

This notebook provides a quick example of how to train a [LightGBM](https://github.com/Microsoft/Lightgbm) model on Spark using [MMLSpark](https://github.com/Azure/mmlspark) for a content-based personalization scenario.

We use the [CRITEO dataset](https://www.kaggle.com/c/criteo-display-ad-challenge), a well known dataset of website ads that can be used to optimize the Click-Through Rate (CTR). The dataset consists of a series of numerical and categorical features and a binary label indicating whether the add has been clicked or not.

The model is based on [LightGBM](https://github.com/Microsoft/Lightgbm), which is a gradient boosting framework that uses tree-based learning algorithms. Finally, we take advantage of
[MMLSpark](https://github.com/Azure/mmlspark) library, which allows LightGBM to be called in a Spark environment and be computed distributely.

This scenario is a good example of **implicit feedback**, where binary labels indicate the interaction between a user and an item. This contrasts with explicit feedback, where the user explicitely rate the content, for example from 1 to 5. 


## Global Settings and Imports

This notebook can be run in a Spark environment in a DSVM or in Azure Databricks. For more details about the installation process, please refer to the [setup instructions](../../SETUP.md).

**NOTE for Azure Databricks:**
* A python script is provided to simplify setting up Azure Databricks with the correct dependencies. Run ```python tools/databricks_install.py -h``` for more details.
* MMLSpark should not be run on a cluster with autoscaling enabled. Disable the flag in the Azure Databricks Cluster configuration before running this notebook.

In [2]:
import os
import sys

sys.path.append("../../")

import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import papermill as pm
import scrapbook as sb

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.common.notebook_utils import is_databricks
from reco_utils.dataset.criteo import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

# Setup MML Spark
if not is_databricks():
    # get the maven coordinates for MML Spark from databricks_install script
    from tools.databricks_install import MMLSPARK_INFO
    packages = [MMLSPARK_INFO["maven"]["coordinates"]]
    repo = MMLSPARK_INFO["maven"].get("repo")
    spark = start_or_get_spark(packages=packages, repository=repo)
    dbutils = None
    print("MMLSpark version: {}".format(MMLSPARK_INFO['maven']['coordinates']))

from mmlspark.train import ComputeModelStatistics
from mmlspark.lightgbm import LightGBMClassifier

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

MMLSpark version: com.microsoft.ml.spark:mmlspark_2.11:0.18.1
System version: 3.6.11 | packaged by conda-forge | (default, Nov 27 2020, 18:57:37) 
[GCC 9.3.0]
PySpark version: 2.4.5


In [3]:
# Criteo data size, it can be "sample" or "full"
DATA_SIZE = "sample"

# LightGBM parameters
# More details on parameters: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
NUM_LEAVES = 32
NUM_ITERATIONS = 50
LEARNING_RATE = 0.1
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 10

# Model name
MODEL_NAME = 'lightgbm_criteo.mml'

## Data Preparation

The [Criteo Display Advertising Challenge](https://www.kaggle.com/c/criteo-display-ad-challenge) (Criteo DAC) dataset is a well-known industry benchmarking dataset for developing CTR prediction models, and is used frequently by research papers. The original dataset contains over 45M rows, but there is also a down-sampled dataset which has 100,000 rows (this can be used by setting `DATA_SIZE = "sample"`). Each row corresponds to a display ad served by Criteo and the first column is indicates whether this ad has been clicked or not.<br><br>
The dataset contains 1 label column and 39 feature columns, where 13 columns are integer values (int00-int12) and 26 columns are categorical features (cat00-cat25).<br><br>
What the columns represent is not provided, but for this case we can consider the integer and categorical values as features representing the user and / or item content. The label is binary and is an example of implicit feedback indicating a user's interaction with an item. With this dataset we can demonstrate how to build a model that predicts the probability of a user interacting with an item based on available user and item content features.


In [4]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, dbutils=dbutils)
# visualize data
raw_data.limit(2).toPandas().head()

100%|██████████| 8.58k/8.58k [00:23<00:00, 371KB/s]


,label,int00,int01,int02,int03,int04,int05,int06,int07,int08,...,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25
0,0,1,1,5,0,1382,4,15,2,181,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,None,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2,0,44,1,102,8,2,2,4,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,None,3a171ecb,43f13e8b,e8b83407,731c3655


### Feature Processing
The feature data provided has many missing values across both integer and categorical feature fields. In addition the categorical features have many distinct values, so effectively cleaning and representing the feature data is an important step prior to training a model.<br><br>
One of the simplest ways of managing both features that have missing values as well as high cardinality is to use the hashing trick. The [FeatureHasher](http://spark.apache.org/docs/latest/ml-features.html#featurehasher) transformer will pass integer values through and will hash categorical features into a sparse vector of lower dimensionality, which can be used effectively by LightGBM.<br><br>
First, the dataset is splitted randomly for training and testing and feature processing is applied to each dataset.

In [5]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [6]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [7]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

In [11]:
train.show(5)

+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------------------+
|label|int00|int01|int02|int03| int04|int05|int06|int07|int08|int09|int10|int11|int12|   cat00|   cat01|   cat02|   cat03|   cat04|   cat05|   cat06|   cat07|   cat08|   cat09|   cat10|   cat11|   cat12|   cat13|   cat14|   cat15|   cat16|   cat17|   cat18|   cat19|   cat20|   cat21|   cat22|   cat23|   cat24|   cat25|            features|
+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-----

In [12]:
train.select("label").groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|18118|
|    0|61992|
+-----+-----+



In [16]:
train.select("features").head(1)

[Row(features=SparseVector(262144, {20766: 1.0, 46747: 18.0, 49602: 1.0, 50388: 1.0, 66551: 0.0, 72199: 1.0, 84264: 29.0, 106531: 1.0, 106700: 1.0, 110783: 1.0, 119348: 1.0, 127003: 4383.0, 130583: 1.0, 134530: 1.0, 135823: 1.0, 138864: 1.0, 142383: 1.0, 151866: 1.0, 156117: 1.0, 164189: 1.0, 169096: 1.0, 190596: 1.0, 209093: 1.0, 212186: 1.0, 215871: 1.0, 220916: 1.0, 236447: 1.0, 239959: 1.0, 252306: 1.0, 258299: 0.0, 261667: -2.0}))]

## Model Training
In MMLSpark, the LightGBM implementation for binary classification is invoked using the `LightGBMClassifier` class and specifying the objective as `"binary"`. In this instance, the occurrence of positive labels is quite low, so setting the `isUnbalance` flag to true helps account for this imbalance.<br><br>

### Hyper-parameters
Below are some of the key [hyper-parameters](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst) for training a LightGBM classifier on Spark:
- `numLeaves`: the number of leaves in each tree
- `numIterations`: the number of iterations to apply boosting
- `learningRate`: the learning rate for training across trees
- `featureFraction`: the fraction of features used for training a tree
- `earlyStoppingRound`: round at which early stopping can be applied to avoid overfitting

In [17]:
lgbm = LightGBMClassifier(
    labelCol="label",
    featuresCol="features",
    objective="binary",
    isUnbalance=True,
    boostingType="gbdt",
    boostFromAverage=True,
    baggingSeed=42,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND
)

### Model Training and Evaluation

In [18]:
model = lgbm.fit(train)

In [19]:
predictions = model.transform(test)

In [20]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("AUC")
)

result = evaluator.transform(predictions)
auc = result.select("AUC").collect()[0][0]
result.show()

+---------------+------------------+
|evaluation_type|               AUC|
+---------------+------------------+
| Classification|0.6874077730353682|
+---------------+------------------+



In [21]:
# Record results with papermill for tests
sb.glue("auc", auc)

## Model Saving 
The full pipeline for operating on raw data including feature processing and model prediction can be saved and reloaded for use in another workflow.

In [22]:
# save model
pipeline = PipelineModel(stages=[feature_processor, model])
pipeline.write().overwrite().save(MODEL_NAME)

In [23]:
# cleanup spark instance
if not is_databricks():
    spark.stop()

## Additional Reading
\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154. https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf <br>
\[2\] MML Spark: https://mmlspark.blob.core.windows.net/website/index.html <br>
